# Neo4j Hello World (Notebook) - Friends Use Case

This notebook connects to a local Neo4j **Community** instance (via Docker), creates a tiny graph, and queries it.

**Assumes** 
 
 
- Neo4j service is running at `bolt://localhost:${URI_PORT}` with the user and password set in the `.env` file. **Run `docker compose up -d`**.
- Ollama service is up on `http://localhost:11434` (ollama default). **Run `ollama serve` and pull the model `ollama pull nomic-embed-text`** (if not pulled yet).



In [23]:
# Dependencies

import os
from dotenv import load_dotenv  
import yaml
from pathlib import Path
from pprint import pprint
from termcolor import cprint

from helpers import helper_folium, helper_leaflet

# === Local services
from neo4j_service import Neo4jService


In [24]:
# Environment variables

load_dotenv()  # Load local environment variables

True

In [25]:
# Load cypher queries

queries = yaml.safe_load(Path("data/friends/queries_friends.yaml").read_text())
queries.keys()  # list available queries

dict_keys(['constraints', 'create_seed', 'show_people', 'show_companies', 'match_adjacency', 'show_text', 'show_locations', 'show_distances', 'add_text', 'add_locations', 'create_vector_indexes', 'delete_all', 'create_person', 'create_company', 'create_person_company_relation', 'create_person_person_relation'])

In [26]:
# Neo4j Langchain wrapper instance
kg = Neo4jService.get_graph()

### 1. Create data

<p align="center">
  <img src="media/KG_step1_populate_graph.svg" width="550">
</p>


- **Entities**: Person and Company nodes with unique constraints (unique name and uuid)
- **Relationships**: KNOWS (person-to-person) and WORKS_AT (person-to-company)
- **Properties**: Basic attributes (name, age, education, industry)


In [27]:
# Populate graph

cprint("\nCreating constraints (if not exist)", "green")
for q in queries["constraints"]:
    kg.query(q)

cprint("\nInit Cleanup.", "green")
for q in queries["delete_all"]:
    kg.query(q)
    
cprint("\nCreate data", "green")
kg.query(queries["create_seed"])



2025-09-30 14:13:00,405 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT person_unique IF NOT EXISTS FOR (e:Person) REQUIRE (e.uuid) IS UNIQUE` has no effect.} {description: `CONSTRAINT person_unique FOR (e:Person) REQUIRE (e.uuid) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT person_unique IF NOT EXISTS\nFOR (p:Person) REQUIRE p.uuid IS UNIQUE\n'
2025-09-30 14:13:00,409 - neo4j.notifications - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT company_unique IF NOT EXISTS FOR (e:Company) REQUIRE (e.uuid) IS UNIQUE` has no effect.} {description: `CONSTRAINT company_unique FOR (e:Company) REQUIRE (e.uuid) IS UNIQUE` already exists.} {position: None} for query: 'CREATE


Creating constraints (if not exist)

Init Cleanup.

Create data


[]

In [28]:
# Example cypher queries

cprint("\nQuery: list all people", "green")
records = kg.query(queries["show_people"]) # <class 'list'>
for r in records:
    print(r)
    
cprint("\nQuery: list all companies", "green")
records = kg.query(queries["show_companies"]) # <class 'list'>
for r in records:
    print(r)

cprint("\nQuery: adjacency (who knows whom)", "green")
records = kg.query(queries["match_adjacency"]) # <class 'list'>
for r in records:
    print(r)


Query: list all people
{'name': 'Paula', 'age': 25, 'p.gender': 'female', 'education': 'Computer Engineering'}
{'name': 'Guillermo', 'age': 26, 'p.gender': 'male', 'education': 'Industrial Engineering'}
{'name': 'Claudia', 'age': 26, 'p.gender': 'female', 'education': 'Physics'}
{'name': 'Iciar', 'age': 26, 'p.gender': 'female', 'education': 'Physics'}
{'name': 'Iria', 'age': 27, 'p.gender': 'female', 'education': 'Physics'}
{'name': 'Marina', 'age': 27, 'p.gender': 'female', 'education': 'Physics'}
{'name': 'Manuel', 'age': 27, 'p.gender': 'male', 'education': 'Arts'}
{'name': 'Xavier', 'age': 27, 'p.gender': 'male', 'education': 'Physics'}
{'name': 'Juan', 'age': 27, 'p.gender': 'male', 'education': 'Physics'}
{'name': 'Aarón', 'age': 27, 'p.gender': 'male', 'education': 'Physics'}
{'name': 'Esteban', 'age': 29, 'p.gender': 'male', 'education': 'Physics'}

Query: list all companies
{'name': 'Waytronics', 'industry': 'Consulting Services'}
{'name': 'Accenture', 'industry': 'Consultin

### 2.A Add rich text info

<p align="center">
  <img src="media/KG_step2_generate_rich_descriptions.svg" width="750">
</p>


In [29]:
# Add rich text descriptions 

cprint("\nQuery: Adding descriptions, appearance and summaries", "green")
for q in queries["add_text"]:
    kg.query(q)



Query: Adding descriptions, appearance and summaries


In [30]:
# Show descriptions
records = kg.query(queries["show_text"])
for r in records:
    print(r)

{'labels(n)': ['Person'], 'n.name': 'Iria', 'n.text': 'Iria is a female of 27 years old and studied Physics.Iria has blue eyes and long brunette and wavy hair. She likes to paint her nails in red or purple colours. She usually wears long earrings.'}
{'labels(n)': ['Person'], 'n.name': 'Guillermo', 'n.text': 'Guillermo is a male of 26 years old and studied Industrial Engineering.Guillermo has brown eyes and short hair. He has a very fancy shirt that he takes to all important events. He shaved his head this summer.'}
{'labels(n)': ['Person'], 'n.name': 'Claudia', 'n.text': "Claudia is a female of 26 years old and studied Physics.Claudia has long curly hair with babylights. She's petite and likes to wear hippie-style clothes."}
{'labels(n)': ['Person'], 'n.name': 'Paula', 'n.text': 'Paula is a female of 25 years old and studied Computer Engineering.Paula short hair in a wolfcut style. She wears long and wide pants and sneakers to the laboratory.'}
{'labels(n)': ['Person'], 'n.name': 'Mari

### 2.B Add location info

In [31]:
# Add location property 

cprint("\nQuery: Adding location property", "green")
for q in queries["add_locations"]:
    kg.query(q)


Query: Adding location property


In [33]:
# Show locations and plot maps
records = kg.query(queries["show_locations"])
# Replace with your query result rows
# records = [
#     {"name":"Iria","lat":40.437596,"lon":-3.711223,"labels":["Person"]},
#     {"name":"Guillermo","lat":40.455022,"lon":-3.692355,"labels":["Person"]},
#     {"name":"Claudia","lat":40.475721,"lon":-3.711451,"labels":["Person"]},
#     {"name":"Paula","lat":40.490170,"lon":-3.654654,"labels":["Person"]},
#     {"name":"Marina"lat":40.367462,"lon":-3.597745,"labels":["Person"]},
#     {"name":"Lumon","lat":40.396648,"lon":-3.624635,"labels":["Company"]},
#     {"name":"CMT","lat":40.453938,"lon":-3.728925,"labels":["Company"]},
#     {"name":"CEBIME","lat":40.549613,"lon":-3.690136,"labels":["Company"]},
# ]
for r in records:
    print(r)


directory=F"{"data/friends/friends_map_"}"
filename=directory+"leaflet.html"
# Follium map
helper_folium.create_map_from_rows(filename, records, center_coordinates=[40.4168, -3.7038])

# Leaflet map
helper_leaflet.create_map_from_rows(filename, records, center_coordinates=[40.4168, -3.7038])

{'name': 'Iria', 'labels': ['Person'], 'lat': 40.4336, 'lon': -3.711223}
{'name': 'Guillermo', 'labels': ['Person'], 'lat': 40.455022, 'lon': -3.692355}
{'name': 'Claudia', 'labels': ['Person'], 'lat': 40.4328, 'lon': -3.711451}
{'name': 'Paula', 'labels': ['Person'], 'lat': 40.49017, 'lon': -3.654654}
{'name': 'Marina', 'labels': ['Person'], 'lat': 40.4332, 'lon': -3.597745}
{'name': 'Manuel', 'labels': ['Person'], 'lat': 40.4336, 'lon': -3.617745}
{'name': 'Xavier', 'labels': ['Person'], 'lat': 40.434, 'lon': -3.711223}
{'name': 'Juan', 'labels': ['Person'], 'lat': 40.4344, 'lon': -3.711223}
{'name': 'Esteban', 'labels': ['Person'], 'lat': 40.4348, 'lon': -3.711223}
{'name': 'Iciar', 'labels': ['Person'], 'lat': 40.4352, 'lon': -3.711223}
{'name': 'Aarón', 'labels': ['Person'], 'lat': 40.4356, 'lon': -3.711223}
{'name': 'Lumon', 'labels': ['Company'], 'lat': 40.533873, 'lon': -3.630539}
{'name': 'CMT', 'labels': ['Company'], 'lat': 40.453938, 'lon': -3.728925}
{'name': 'CEBIME', 'lab

### 3. Create property embeddings (first step into RAG) 

<p align="center">
  <img src="media/KG_step3_generate_property_embeddings.svg" width="750">
</p>

**RAG** implementation requires selecting a **property to embed and use for similarity searches**. 

Description properties containing **rich text** work well for this purpose, as they provide richer semantic information. In our example, we'll use *text*.

In order to do so, we create two vector indexes in Neo4j:

- **Vector index *person_node_info_idx***: based on property ***info_emb*** for nodes of type "Person"
-  **Vector index *company_node_info_idx***: based on property ***info_emb*** for nodes of type "Company"

After that, we create the embeddings (this happens for both Person nodes and Company nodes):

- **Property *text*** ---`nomic-embed-text`---> **Property *embedding***


In [34]:
# Create vector indexes

for q in queries["create_vector_indexes"]:
    kg.query(q)

# Show created vector indexes
results = kg.query("SHOW VECTOR INDEXES")
idx = list(results)
cprint(f"\nFound {len(idx)} vector index entries.", "green")
for r in idx:
    cprint("-"*20,"green")
    pprint(r)


Found 4 vector index entries.
--------------------
{'entityType': 'NODE',
 'id': 5,
 'indexProvider': 'vector-2.0',
 'labelsOrTypes': ['Chunk'],
 'lastRead': neo4j.time.DateTime(2025, 9, 30, 12, 9, 22, 479000000, tzinfo=<UTC>),
 'name': 'chunks_node_idx',
 'owningConstraint': None,
 'populationPercent': 100.0,
 'properties': ['embedding'],
 'readCount': 1,
 'state': 'ONLINE',
 'type': 'VECTOR'}
--------------------
{'entityType': 'NODE',
 'id': 8,
 'indexProvider': 'vector-2.0',
 'labelsOrTypes': ['Company'],
 'lastRead': None,
 'name': 'company_node_idx',
 'owningConstraint': None,
 'populationPercent': 100.0,
 'properties': ['embedding'],
 'readCount': None,
 'state': 'ONLINE',
 'type': 'VECTOR'}
--------------------
{'entityType': 'RELATIONSHIP',
 'id': 9,
 'indexProvider': 'vector-2.0',
 'labelsOrTypes': ['KNOWS'],
 'lastRead': None,
 'name': 'know_relationship_idx',
 'owningConstraint': None,
 'populationPercent': 100.0,
 'properties': ['embedding'],
 'readCount': None,
 'state':

In [35]:
# Create property embeddings 

# (p:PERSON): create embeddings only for nodes missing them
Neo4jService.vectorize_property(
                   element = "node", 
                   node_label = "Person",
                   source_property = "text"
                   )

# (c:COMPANY): create embeddings only for nodes missing them
Neo4jService.vectorize_property(
                   element = "node", 
                   node_label = "Company", 
                   source_property = "text",
                   )

# [r:KNOWS]: create embeddings only for nodes missing them
Neo4jService.vectorize_property(
                   element = "relationship",
                   rel_type = "KNOWS",
                   source_property = "text"
                   )

2025-09-30 14:13:11,011 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:13:11,056 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"



Generating embeddings for (n:Person) on n.text
 Updated 1 embeddings
 Updated 2 embeddings


2025-09-30 14:13:11,084 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:13:11,113 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:13:11,142 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


 Updated 3 embeddings
 Updated 4 embeddings
 Updated 5 embeddings


2025-09-30 14:13:11,184 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:13:11,215 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:13:11,246 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:13:11,274 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


...

 Updated 55 embeddings
 Updated 56 embeddings
 Updated 57 embeddings
 Updated 58 embeddings


2025-09-30 14:13:13,573 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:13:13,602 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:13:13,627 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:13:13,656 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


 Updated 59 embeddings
 Updated 60 embeddings
 Updated 61 embeddings
 Updated 62 embeddings


2025-09-30 14:13:13,680 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-09-30 14:13:13,708 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


 Updated 63 embeddings
 Updated 64 embeddings


### 4. Search 

Whenever we query this graph, we can use two different but complementary search techniques:

1. **KG Retreival**: through **Neo4J Cypher Query Language (CQL)** we can query precise entities and relations. The input query must be translated into CQL to get the desired results.

2. **Vector Retrieval**: **embedding the input query**, we can make a vector search against the vector indexes defined above.

The results will be a combination of both searches.

<p align="center">
  <img src="media/KGRAG_schema.svg">
</p>


In [36]:
# KG RAG Search

# Query Nodes
result = Neo4jService.neo4j_KGRAG_search(
                             query = "Who shaved its head this summer?", 
                             index = "person_node_idx",
                             source_property = "text",
                             main_property = "name",
                             top_k = 5
                             )
pprint(result, width = 200, sort_dicts=False, indent=2)
file = "data/friends/friends_context_1.txt"
with open(file, 'w', encoding='utf-8') as f:
  f.write(result.get("combined_context", ""))

result  = Neo4jService.neo4j_KGRAG_search(
                                query = "Which company investigates Cancer?",
                                index = "company_node_idx",
                                source_property = "text",
                                main_property = "name",
                                top_k = 5
                              )
pprint(result, width = 200, sort_dicts=False, indent=2)
file = "data/friends/friends_context_2.txt"
with open(file, 'w', encoding='utf-8') as f:
  f.write(result.get("combined_context", ""))

2025-09-30 14:13:13,739 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"



Running vector search query to retrieve context.


2025-09-30 14:13:13,889 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


{ 'query': 'Who shaved its head this summer?',
  'total_results': 5,
  'raw_search_results': [ { 'score': 0.837460994720459,
                            'label': ['Person'],
                            'properties_dict': { 'text': 'Guillermo is a male of 26 years old and studied Industrial Engineering.Guillermo has brown eyes and short hair. He has a very fancy shirt '
                                                         'that he takes to all important events. He shaved his head this summer.',
                                                 'location': POINT(-3.692355 40.455022),
                                                 'name': 'Guillermo',
                                                 'age': 26,
                                                 'gender': 'male',
                                                 'education': 'Industrial Engineering'},
                            'facts': [ 'Guillermo -[WORKS_AT {"since":2023}]-> Lumon',
                                   

In [37]:
# Query Relationships
result  = Neo4jService.neo4j_KGRAG_search(
                              query = "Who is helping Iria at work?",
                              index = "know_relationship_idx",
                              source_property = "text",
                              main_property = "name",
                              top_k = 5
                              )
pprint(result, width = 200, sort_dicts=False)

file = "data/friends/friends_context_3.txt"
with open(file, 'w', encoding='utf-8') as f:
  f.write(result.get("combined_context", ""))


2025-09-30 14:13:13,948 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"



Running vector search query to retrieve context.
{'query': 'Who is helping Iria at work?',
 'total_results': 5,
 'raw_search_results': [{'score': 0.9195809364318848,
                         'type': 'KNOWS',
                         'properties_dict': {'text': 'Paula is helping Iria with the project she is working on.', 'knows_from': 'work'},
                         'facts': ['Iria -[KNOWS {"knows_from":"work","text":"Paula is helping Iria with the project she is working on."}]-> Paula']},
                        {'score': 0.8711152076721191,
                         'type': 'KNOWS',
                         'properties_dict': {'text': 'Guillermo and Iria work together in the same project with AI Agents.', 'knows_from': 'work'},
                         'facts': ['Iria -[KNOWS {"knows_from":"work","text":"Guillermo and Iria work together in the same project with AI Agents."}]-> Guillermo']},
                        {'score': 0.8619155883789062,
                         'type': 'KNOWS